# モデルチューニング

##scikit-learnの各種モデル利用

In [1]:
import numpy as np
import pandas as pd
import os
import pickle
import gc 

# 分布確認  anacondaでは、一先ず、コメントにする。
#import pandas_profiling as pdp

# 可視化
import matplotlib.pyplot as plt

# 前処理
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder

# バリデーション
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold

# 評価指標
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix

# モデリング: lightgbm
import lightgbm as lgb

import warnings
warnings.filterwarnings("ignore")

# matplotilbで日本語表示したい場合はこれをinstallしてインポートする
!pip install japanize-matplotlib
import japanize_matplotlib
%matplotlib inline

#### ファイル読み込みとデータセット作成

In [2]:
# ファイル読み込み
df_train = pd.read_csv("Bank Churn/train.csv")

#特徴量エンジニアリング
df_train["NumOfProducts_/_Age"] = df_train["NumOfProducts"] / df_train["Age"]

# BalanceとNumOfProductsの四則計算の特徴量作成　（数値変数×数値変数）
df_train["Balance_-_NumOfProducts"] = df_train["Balance"] - df_train["NumOfProducts"]
# HasCrCardとNumOfProductsの平均値生成　(数値変数xカテゴリ変数)
df_train["mean_NumOfProducts_by_HasCrCard"] = df_train.groupby("NumOfProducts")["HasCrCard"].transform("mean")



In [3]:
### OneHotEncoderのインスタンスを作成し、sparse=Falseで密行列を指定
ohe_Gender=OneHotEncoder(sparse=False)

### OneHotEncoderを訓練データの"Gendet"列に適させる
ohe_Gender.fit(df_train[["Gender"]])
### OneHotエンコーディングされた特徴量を含むDataFrameを作成し、列名を指定
tmp_Gender = pd.DataFrame(
    ### OneHotエンコーディングを実行し、結果をDataFrameに保存
    ohe_Gender.transform(df_train[["Gender"]]), 
    ### 列名を "Embarked_0", "Embarked_1" などに設定
    columns=["Gender_{}".format(i) for i in ohe_Gender.categories_[0]],
)

### 元のデータフレームとOneHotエンコーディングされた特徴量を結合
df_train = pd.concat([df_train, tmp_Gender], axis=1)

### "Embarked"列とOne-Hotエンコーディングされた列を抽出して表示
df_train[["Gender","Gender_Female","Gender_Male"]]

,Gender,Gender_Female,Gender_Male
0,Male,0.0,1.0
1,Male,0.0,1.0
2,Male,0.0,1.0
3,Male,0.0,1.0
4,Male,0.0,1.0
...,...,...,...
165029,Female,1.0,0.0
165030,Male,0.0,1.0
165031,Male,0.0,1.0
165032,Female,1.0,0.0


In [4]:
##特徴量追加　　数値×数値
# Gender_MaleとIsActiveMemberを組み合わせて新しい特徴量を生成(加)
df_train["Gender_Male_+_Active"] = df_train["Gender_Male"] + df_train["IsActiveMember"]

# 新しい特徴量の確認
print(df_train[["Gender_Male", "IsActiveMember", "Gender_Male_+_Active"]].head())

   Gender_Male  IsActiveMember  Gender_Male_+_Active
0          1.0             0.0                   1.0
1          1.0             1.0                   2.0
2          1.0             0.0                   1.0
3          1.0             1.0                   2.0
4          1.0             1.0                   2.0


In [5]:

# データセット作成
x_train = df_train[["IsActiveMember","EstimatedSalary","Age","NumOfProducts","NumOfProducts_/_Age","Balance","Balance_-_NumOfProducts",
                    "mean_NumOfProducts_by_HasCrCard","Gender_Male_+_Active"]]
y_train = df_train[["Exited"]]


#### 数値データの正規化

In [6]:
##上の追加した特徴量の正規化から
### "数値データの特徴量列の値を最小値で引き、その結果を"数値データの特徴量列の最大値と最小値の差で割ることで、最小-最大スケーリングを実行
x_train["IsActiveMember"] = (x_train["IsActiveMember"] - x_train["IsActiveMember"].min()) / (x_train["IsActiveMember"].max() - x_train["IsActiveMember"].min()) 

x_train["EstimatedSalary"] = (x_train["EstimatedSalary"] - x_train["EstimatedSalary"].min()) / (x_train["EstimatedSalary"].max() - x_train["EstimatedSalary"].min())

x_train["Age"] = (x_train["Age"] - x_train["Age"].min()) / (x_train["Age"].max() - x_train["Age"].min())

x_train["NumOfProducts"] = (x_train["NumOfProducts"] - x_train["NumOfProducts"].min()) / (x_train["NumOfProducts"].max() - x_train["NumOfProducts"].min())

x_train["NumOfProducts_/_Age"] = (x_train["NumOfProducts_/_Age"] - x_train["NumOfProducts_/_Age"].min()) / (x_train["NumOfProducts_/_Age"].max() - x_train["NumOfProducts_/_Age"].min())
x_train["Balance"] = (x_train["Balance"] - x_train["Balance"].min()) / (x_train["Balance"].max() - x_train["Balance"].min())

x_train["Balance_-_NumOfProducts"] = (x_train["Balance_-_NumOfProducts"] - x_train["Balance_-_NumOfProducts"].min()) / (x_train["Balance_-_NumOfProducts"].max() - x_train["Balance_-_NumOfProducts"].min())

x_train["mean_NumOfProducts_by_HasCrCard"] = (x_train["mean_NumOfProducts_by_HasCrCard"] - x_train["mean_NumOfProducts_by_HasCrCard"].min()) / (x_train["mean_NumOfProducts_by_HasCrCard"].max() - x_train["mean_NumOfProducts_by_HasCrCard"].min())

x_train["Gender_Male_+_Active"] = (x_train["Gender_Male_+_Active"] - x_train["Gender_Male_+_Active"].min()) / (x_train["Gender_Male_+_Active"].max() - x_train["Gender_Male_+_Active"].min())



#### 学習データと検証データの分割（ホールドアウト検証）

In [7]:
x_tr, x_va, y_tr, y_va = train_test_split(x_train, y_train, test_size=0.2,stratify=y_train,random_state=123)
print(x_tr.shape, x_va.shape, y_tr.shape, y_va.shape)

(132027, 9) (33007, 9) (132027, 1) (33007, 1)


# モデル学習　　必要に応じて以下のモデルを使用する。

#### LogisticRegression(線形回帰)

In [8]:
### scikit-learnライブラリからロジスティック回帰モデルをインポート
from sklearn.linear_model import LogisticRegression

### ロジスティック回帰モデルのインスタンスを作成し、変数model_logisに代入
model_logis = LogisticRegression()

### 訓練データ x_tr とそのラベル y_tr を使ってロジスティック回帰モデルを学習
model_logis.fit(x_tr, y_tr)

### 学習済みのロジスティック回帰モデルを使って検証データ x_va の予測を行い、予測結果を y_va_pred に格納
y_va_pred = model_logis.predict_proba(x_va)[:,1]

### 予測の精度を計算し表示。accuracy_score関数は予測結果と正解ラベルの一致率を計算
print("accuracy:{:.4f}".format(roc_auc_score(y_va, y_va_pred)))

### 検証データの最初の5つのサンプルに対する予測結果を表示
print(y_va_pred[:5])

accuracy:0.8563
[0.71531069 0.11442243 0.2120805  0.04526581 0.11609275]


In [ ]:
#### 確率値の取得

In [9]:
### ロジスティック回帰モデルを使って、検証データセット x_va に対する各サンプルのクラスの予測確率を計算
y_va_pred_proba = model_logis.predict_proba(x_va)[:, 1]

### 最初の5つのサンプルの予測確率を取得し、出力
print(y_va_pred_proba[:5])

[0.71531069 0.11442243 0.2120805  0.04526581 0.11609275]


##分類：ランダムフォレスト

In [10]:
from sklearn.ensemble import RandomForestClassifier

# ランダムフォレストの学習モデルを作る（訓練データで）
model_logis = RandomForestClassifier()
model_logis.fit(x_tr, y_tr)

# 正解率を調べる（テストデータで）
y_va_pred = model_logis.predict_proba(x_va)[:,1]


### 予測の精度を計算し表示。accuracy_score関数は予測結果と正解ラベルの一致率を計算
print("accuracy:{:.4f}".format(roc_auc_score(y_va, y_va_pred)))

### 検証データの最初の5つのサンプルに対する予測結果を表示
print(y_va_pred[:5])



accuracy:0.8331
[0.23 0.32 0.24 0.01 0.  ]


##決定木

In [11]:
from sklearn.tree import DecisionTreeClassifier


# 決定木の学習モデルを作る（訓練データで）
model_logis = DecisionTreeClassifier(max_depth=None, random_state=0)
model_logis.fit(x_tr, y_tr)

# 正解率を調べる（テストデータで）
y_va_pred = model_logis.predict_proba(x_va)[:,1]

### 予測の精度を計算し表示。accuracy_score関数は予測結果と正解ラベルの一致率を計算
print("accuracy:{:.4f}".format(roc_auc_score(y_va, y_va_pred)))

### 検証データの最初の5つのサンプルに対する予測結果を表示
print(y_va_pred[:5])



accuracy:0.6960
[1. 1. 0. 0. 0.]


#### 確率値の取得

In [12]:
### 検証データセット x_va に対する各サンプルのクラスの予測確率を計算
y_va_pred_proba = model_logis.predict_proba(x_va)[:, 1]

### 最初の5つのサンプルの予測確率を取得し、出力
print(y_va_pred_proba[:5])

[1. 1. 0. 0. 0.]


##推論用ファイル読み込みとデータセットを作成

In [13]:
# ファイル読み込み
df_test = pd.read_csv("Bank Churn/test.csv")

#特徴量エンジニアリング
df_test["NumOfProducts_/_Age"] = df_test["NumOfProducts"] / df_test["Age"]

# BalanceとNumOfProductsの四則計算の特徴量作成　（数値変数×数値変数）
df_test["Balance_-_NumOfProducts"] = df_test["Balance"] - df_test["NumOfProducts"]

# HasCrCardとNumOfProductsの平均値生成　(数値変数xカテゴリ変数)
df_test["mean_NumOfProducts_by_HasCrCard"] = df_test.groupby("NumOfProducts")["HasCrCard"].transform("mean")



In [14]:
### OneHotEncoderのインスタンスを作成し、sparse=Falseで密行列を指定
ohe_Gender=OneHotEncoder(sparse=False)

### OneHotEncoderを訓練データの"Gendet"列に適させる
ohe_Gender.fit(df_test[["Gender"]])
### OneHotエンコーディングされた特徴量を含むDataFrameを作成し、列名を指定
tmp_Gender = pd.DataFrame(
    ### OneHotエンコーディングを実行し、結果をDataFrameに保存
    ohe_Gender.transform(df_test[["Gender"]]), 
    ### 列名を "Embarked_0", "Embarked_1" などに設定
    columns=["Gender_{}".format(i) for i in ohe_Gender.categories_[0]],
)

### 元のデータフレームとOneHotエンコーディングされた特徴量を結合
df_test = pd.concat([df_test, tmp_Gender], axis=1)

### "Embarked"列とOne-Hotエンコーディングされた列を抽出して表示
df_test[["Gender","Gender_Female","Gender_Male"]]

,Gender,Gender_Female,Gender_Male
0,Female,1.0,0.0
1,Female,1.0,0.0
2,Female,1.0,0.0
3,Male,0.0,1.0
4,Male,0.0,1.0
...,...,...,...
110018,Male,0.0,1.0
110019,Female,1.0,0.0
110020,Male,0.0,1.0
110021,Female,1.0,0.0


In [15]:
##特徴量追加　　数値×数値
# Gender_MaleとIsActiveMemberを組み合わせて新しい特徴量を生成(加)
df_test["Gender_Male_+_Active"] = df_test["Gender_Male"] + df_test["IsActiveMember"]

# 新しい特徴量の確認
print(df_test[["Gender_Male", "IsActiveMember", "Gender_Male_+_Active"]].head())

   Gender_Male  IsActiveMember  Gender_Male_+_Active
0          0.0             1.0                   1.0
1          0.0             0.0                   0.0
2          0.0             0.0                   0.0
3          1.0             0.0                   1.0
4          1.0             0.0                   1.0


In [16]:
# データセット作成
x_test = df_test[["IsActiveMember","EstimatedSalary","Age","NumOfProducts","NumOfProducts_/_Age","Balance",
                  "Balance_-_NumOfProducts","mean_NumOfProducts_by_HasCrCard","Gender_Male_+_Active"]]
id_test = df_test[["id"]]

#"Balance_-_NumOfProducts"

##推論用　数値データの正規化

In [17]:
x_test["IsActiveMember"] = (x_test["IsActiveMember"] - x_test["IsActiveMember"].min()) / (x_test["IsActiveMember"].max() - x_test["IsActiveMember"].min()) 

x_test["EstimatedSalary"] = (x_test["EstimatedSalary"] - x_test["EstimatedSalary"].min()) / (x_test["EstimatedSalary"].max() - x_test["EstimatedSalary"].min())

x_test["Age"] = (x_test["Age"] - x_test["Age"].min()) / (x_test["Age"].max() - x_test["Age"].min())

x_test["NumOfProducts"] = (x_test["NumOfProducts"] - x_test["NumOfProducts"].min()) / (x_test["NumOfProducts"].max() - x_test["NumOfProducts"].min())

x_test["NumOfProducts_/_Age"] = (x_test["NumOfProducts_/_Age"] - x_test["NumOfProducts_/_Age"].min()) / (x_test["NumOfProducts_/_Age"].max() - x_test["NumOfProducts_/_Age"].min())

x_test["Balance"] = (x_test["Balance"] - x_test["Balance"].min()) / (x_test["Balance"].max() - x_test["Balance"].min())

x_test["Balance_-_NumOfProducts"] = (x_test["Balance_-_NumOfProducts"] - x_test["Balance_-_NumOfProducts"].min()) / (x_test["Balance_-_NumOfProducts"].max() - x_test["Balance_-_NumOfProducts"].min())

x_test["mean_NumOfProducts_by_HasCrCard"] = (x_test["mean_NumOfProducts_by_HasCrCard"] - x_test["mean_NumOfProducts_by_HasCrCard"].min()) / (x_test["mean_NumOfProducts_by_HasCrCard"].max() - x_test["mean_NumOfProducts_by_HasCrCard"].min())
x_test["Gender_Male_+_Active"] = (x_test["Gender_Male_+_Active"] - x_test["Gender_Male_+_Active"].min()) / (x_test["Gender_Male_+_Active"].max() - x_test["Gender_Male_+_Active"].min())



## テストデータの予測（確率値を取得)

In [18]:
y_test_pred = model_logis.predict_proba(x_test)

## 提出用ファイルの作成

In [19]:
df_submit = pd.DataFrame({"id": id_test["id"], "Exited": y_test_pred[:, 1]})
display(df_submit.head(5))
df_submit.to_csv("submission_scikit_LoRe.csv", index=None)

,id,Exited
0,165034,0.0
1,165035,1.0
2,165036,0.0
3,165037,1.0
4,165038,0.0


#### SVM (実施保留中)　　

In [ ]:
### scikit-learnライブラリからSVC（サポートベクトル分類器）クラスをインポートし、SVMモデルのインスタンスを作成
from sklearn.svm import SVC

model_svm = SVC(C=1.0, random_state=123, probability=True)

### 訓練データ x_tr とそのラベル y_tr を使ってSVMモデルを学習
model_svm.fit(x_tr, y_tr)

### 学習済みのSVMモデルを使って検証データ x_va の予測を行い、予測結果を y_va_pred に格納
y_va_pred = model_svm.predict_proba(x_va)[:,1]

### 予測の精度を計算し表示。accuracy_score関数は予測結果と正解ラベルの一致率を計算
print("accuracy:{:.4f}".format(roc_auc_score(y_va, y_va_pred)))


### 学習済みのSVMモデルを使って検証データ x_va の各サンプルに対するクラスの予測確率を計算し表示
y_va_pred_proba = model_svm.predict_proba(x_va)[:,1]
print(y_va_pred_proba[:5])